# Gravity Model PPML Estimation

## Formalization of the gravity model equation
### with origin-destination (importer-exporter) fixed effects and lagged independent variables

\begin{equation}
\begin{split}
EXPORT_{ij,t} =\; &\exp\Bigl[\hat{\alpha}_{ij}
+ \sum_{k=1}^5 \hat{\beta}_{1k}\,GDP_{i,t-k}
+ \sum_{l=1}^5 \hat{\beta}_{2l}\,GDP_{j,t-l} \\
&\quad
+ \sum_{m=1}^5 \hat{\beta}_{3m}\,SANCT_{ij,t-m}\Bigr]
\times \epsilon_{ij,t}
\end{split}
\end{equation}

Where:
- $\hat\alpha_{ij}$ - time-invariant fixed effects
- $GDP_{i,t}$ - size of origin country $i$ at time $t$
- $GDP_{j,t}$ - size of destination country $j$ at time $t$
- $SANCT_{ij,t}$ - unilateral sanction relationship $i \rightarrow j$ at time $t$
- $\hat\beta_{1k}$, $\hat\beta_{2l}$, $\hat\beta_{3m}$ - estimated coefficients for the respective lagged variables
- $\epsilon_{ij,t}$ - error term


In [151]:
import gme
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy import sparse
from sklearn.model_selection import GroupShuffleSplit

import thesis_utils as tu

In [152]:
# Config for saving outputs
SAVE_ENABLED = True
SERIAL_NUMBER = "NOT_SET"
VERBOSE = 0

# Model parameters
N_LAGS = 5
N_DYADS = 5000
MAX_ITERATIONS = 5000
SUBSAMPLE_ENABLED = False

# Train parameters
TARGET = "EXPORT"
RANDOM_SEED = 16

SANCTION_COLS = ["arms", "military", "trade", "travel", "other"]

In [153]:
processed = pd.read_parquet(path="../../data/model/processed.parquet", engine="fastparquet")
df: DataFrame = processed.copy(deep=True)

In [154]:
# Sort data by Report + Partner + Year
df["dyad_id"] = df["ISO3_reporter"] + "_" + df["ISO3_partner"]
df = df.sort_values(by=["dyad_id", "Year"]).reset_index(drop=True)

In [155]:
if SUBSAMPLE_ENABLED:
  dyad_subsample = pd.Series(df["dyad_id"].unique()).sample(n=N_DYADS, random_state=RANDOM_SEED, replace=False)
  df = df[df["dyad_id"].isin(dyad_subsample)]
print(df["dyad_id"].nunique())
df["sanction"] = (df[SANCTION_COLS]
                  .sum(axis=1)).astype(int)

33672


In [156]:
lag_cols = ["GDP_reporter", "GDP_partner", "sanction"]
invariant_cols = ["contig", "comlang_off", "colony", "smctry", "distw"]

In [157]:
# cont_vars = lag_cols
# p = len(cont_vars)
# mu = df[cont_vars].mean()
#
# # 3. subtract dyad‐specific means
# df_within = df.copy()
# df_within[cont_vars] = (df_within.groupby('dyad_id')[cont_vars]
#                         .transform(lambda x: x - x.mean()))
#
# # 4. add back global mean * (p - 1)
# df_within[cont_vars] += mu * (p - 1)
# df = df_within.copy()

In [158]:
all_cols = lag_cols + [TARGET]

# 1) compute each row’s dyad mean for those columns
mean_data = df.groupby("dyad_id")[all_cols].transform("mean")

# 2) subtract it in‐place
df[all_cols] = df[all_cols] - mean_data

In [159]:
df["GDP_reporter"] = np.log1p(df["GDP_reporter"]).astype(float)
df["GDP_partner"] = np.log1p(df["GDP_partner"]).astype(float)
df["EXPORT"] = np.log1p(df["EXPORT"])
df["distw"] = np.log1p(df["distw"]).astype(float)

/opt/homebrew/Caskroom/miniconda/base/envs/thesis_env/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/thesis_env/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/thesis_env/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [160]:
# Coerce numerical values
num_cols = ["distw", "GDP_reporter", "GDP_partner", "sanction", "contig",
            "comlang_off", "colony", "smctry", "Year", ]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce").astype(float)
df = df.dropna(subset=num_cols)

In [161]:
for col in lag_cols:
  for index in range(1, N_LAGS + 1):
    df[f"{col}_lag{index}"] = df.groupby("dyad_id")[col].shift(index)

df = df.dropna()

df["Year"] = df["Year"].astype(int)
for col in ["dyad_id"]:
  df[col] = pd.Categorical(df[col], categories=sorted(df[col].unique()))
df[("dyad_id")] = df["dyad_id"].astype("category").cat.codes

In [162]:
# df = tu.add_fixed_effects(df)

In [163]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)

train_df_idx, test_df_idx = next(gss.split(df, groups=df["dyad_id"]))
test_df = df.iloc[test_df_idx]
train_df = df.iloc[train_df_idx]

In [164]:
rhs_cols = [f"{c}_lag{index}" for c in lag_cols for index in range(1, N_LAGS + 1)]
fe_columns = [col for col in train_df.columns if col.startswith("fe_dyad_id_")]
convert_to_float_columns = rhs_cols + invariant_cols
train_df.loc[:, convert_to_float_columns] = train_df.loc[:, convert_to_float_columns].astype(
  "float32",
  copy=False
)
rhs_cols = rhs_cols + invariant_cols + fe_columns

print("Fixed effects columns:", fe_columns[:10])
print("Lagged columns:", rhs_cols[:10])

Fixed effects columns: []
Lagged columns: ['GDP_reporter_lag1', 'GDP_reporter_lag2', 'GDP_reporter_lag3', 'GDP_reporter_lag4', 'GDP_reporter_lag5', 'GDP_partner_lag1', 'GDP_partner_lag2', 'GDP_partner_lag3', 'GDP_partner_lag4', 'GDP_partner_lag5']


In [165]:
# Prepare data
print("Selecting columns based on rhs_cols...")
col_idx = [train_df.columns.get_loc(c) for c in rhs_cols]
print("Finished extracting raw features.")

Selecting columns based on rhs_cols...
Finished extracting raw features.


In [166]:
print("Extracting features from train_df...", flush=True)
X = train_df.iloc[:, col_idx].to_numpy(dtype=float, copy=False)
print("Finished extracting features.")
print("Creating sparse matrix from features...")
X = sparse.csr_matrix(X, dtype=float)
print("Finished creating sparse matrix from features.")

Extracting features from train_df...
Finished extracting features.
Creating sparse matrix from features...
Finished creating sparse matrix from features.


In [167]:
y = train_df["EXPORT"].to_numpy(dtype=np.float32)
print("Finished extracting target variable.")

Finished extracting target variable.


In [168]:
# 1) Cross-validated Lasso to pick alpha
# lasso_cv = LassoCV(cv=5, max_iter=MAX_ITERATIONS, tol=1e-4,
#                    verbose=VERBOSE, n_jobs=-1)
# print("Fitting celer.LassoCV (5-fold, all CPUs)…")
# lasso_cv.fit(X, y)
# cv_coef = lasso_cv.coef_

In [169]:
# Scale down the chosen alpha if desired
# best_alpha = lasso_cv.alpha_
# print(f"Selected alpha from CV: {best_alpha:.3e}")

In [170]:
# 2) Build threshold grid from non-zero CV coefficients
# nz = np.abs(cv_coef[cv_coef != 0])
# if nz.size == 0:
#   print("Warning: no non-zero coefficients found! Using fallback.")
#   nz = np.array([1e-5])
# min_c, max_c = nz.min(), nz.max()
# threshold_grid = np.logspace(np.log10(min_c / 10),
#                              np.log10(max_c / 10), 10)
# print("Threshold grid:", threshold_grid)
#
# # 3) Fit final Lasso at best_alpha
# print("Fitting final celer.Lasso…")
# lasso_full = Lasso(alpha=best_alpha, max_iter=MAX_ITERATIONS,
#                    tol=1e-4,
#                    verbose=VERBOSE)
# lasso_full.fit(X, y)
# full_coef = lasso_full.coef_

In [171]:
# 4) Loop over thresholds, refit on selected features, and compute RMSE
# results_threshold = {
#   "thresholds": [],
#   "errors": [],
#   "n_columns": [],
#   "selected_columns": []
# }
#
# for thr in threshold_grid:
#   print(f"\nThreshold {thr:.5f}")
#   idx = np.where(np.abs(full_coef) >= thr)[0]
#   cols = [rhs_cols[i] for i in idx]
#   print(f"  -> {len(cols)} features selected")
#
#   Xr = X[:, idx]
#   lasso_sub = Lasso(alpha=best_alpha, max_iter=MAX_ITERATIONS,
#                     tol=1e-4, verbose=VERBOSE)
#   lasso_sub.fit(Xr, y)
#   y_pred = lasso_sub.predict(Xr)
#   err = tu.rmse(y, y_pred)
#
#   results_threshold["thresholds"].append(thr)
#   results_threshold["errors"].append(err)
#   results_threshold["n_columns"].append(len(cols))
#   results_threshold["selected_columns"].append(cols)

In [172]:
# best_idx = np.argmin(results_threshold["errors"])
#
# best_rmse = results_threshold["errors"][best_idx]
# best_columns = results_threshold["selected_columns"][best_idx]
#
# print(f"Best RMSE: {best_rmse}")
# print(f"Best columns ({len(best_columns)}):")
# print(best_columns)

In [173]:
train_df.loc[:, 'ISO3_reporter'] = pd.Series(train_df['ISO3_reporter'].values, index=train_df.index).astype(str)
train_df.loc[:, 'ISO3_partner'] = pd.Series(train_df['ISO3_partner'].values, index=train_df.index).astype(str)
train_df.loc[:, 'Year'] = pd.to_numeric(train_df['Year'], errors='coerce')

In [174]:
for col in rhs_cols:
  if col == "dyad_id":
    continue
  train_df.loc[:, col] = pd.to_numeric(train_df[col], errors='coerce', downcast="float")

In [175]:
train_df.columns

Index(['ISO3_reporter', 'UNDS_reporter', 'CNAME_reporter', 'ISO3_partner',
       'UNDS_partner', 'CNAME_partner', 'Year', 'GDP_reporter', 'GDP_partner',
       'contig', 'comlang_off', 'comlang_ethno', 'colony', 'smctry', 'distcap',
       'distw', 'IMPORT', 'EXPORT', 'arms', 'military', 'trade', 'descr_trade',
       'financial', 'travel', 'other', 'target_mult', 'sender_mult',
       'GDP_yearly_average', 'dyad_id', 'sanction', 'GDP_reporter_lag1',
       'GDP_reporter_lag2', 'GDP_reporter_lag3', 'GDP_reporter_lag4',
       'GDP_reporter_lag5', 'GDP_partner_lag1', 'GDP_partner_lag2',
       'GDP_partner_lag3', 'GDP_partner_lag4', 'GDP_partner_lag5',
       'sanction_lag1', 'sanction_lag2', 'sanction_lag3', 'sanction_lag4',
       'sanction_lag5'],
      dtype='object')

In [176]:
# Create estimation object
gme_data = gme.EstimationData(
  data_frame=train_df,
  imp_var_name="ISO3_reporter",
  exp_var_name="ISO3_partner",
  output_var_name=TARGET,
  year_var_name="Year"
)
gme_data

number of countries: 179 
number of exporters: 179 
number of importers: 179 
number of years: 15 
number of sectors: not_applicable 
dimensions: (166398, 45)

In [177]:
# Create GME model
gme_model = gme.EstimationModel(
  estimation_data=gme_data,
  lhs_var=TARGET,
  rhs_var=rhs_cols,
  drop_intratrade=True,
)

In [178]:
# Conduct PPML estimation of coefficients
estimates = gme_model.estimate()

Estimation began at 01:30 PM  on Jun 30, 2025
Omitted Regressors: []
Estimation completed at 01:30 PM  on Jun 30, 2025


In [179]:
gme_model.ppml_diagnostics

Overfit Warning                             No
Number of Regressors Dropped                 0
Regressors with Zero Trade                  []
Regressors from User                        []
Regressors Perfectly Collinear              []
Completion Time                   0.01 minutes
dtype: object

In [180]:
# estimates.keys()
results = estimates['all']
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 EXPORT   No. Observations:               166398
Model:                            GLM   Df Residuals:                   166378
Model Family:                 Poisson   Df Model:                           19
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -5.1450e+05
Date:                Mon, 30 Jun 2025   Deviance:                   1.1714e+06
Time:                        13:30:24   Pearson chi2:                 4.71e+05
No. Iterations:                     6   Pseudo R-squ. (CS):             0.9445
Covariance Type:                  HC1                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
GDP_reporter_lag1     0.1334      0.006     24.185      0.000       0.123       0.144
GDP_reporter_lag2     0.0140      0.007      1.916      0.055      -0.000       0.028
GDP_reporter_lag3     0.0163      0.007      2.286      0.022       0.002       0.030
GDP_reporter_lag4     0.0142      0.006      2.427      0.015       0.003       0.026
GDP_reporter_lag5    -0.0011      0.003     -0.324      0.746      -0.008       0.006
GDP_partner_lag1      0.0379      0.005      8.306      0.000       0.029       0.047
GDP_partner_lag2      0.0195      0.006      3.180      0.001       0.007       0.031
GDP_partner_lag3      0.0211      0.006      3.393      0.001       0.009       0.033
GDP_partner_lag4      0.0021      0.005      0.387      0.699      -0.008       0.012
GDP_partner_lag5     -0.0093      0.003     -2.914      0.004      -0.016      -0.003
sanction_lag1         0.0333      0.006      5.609      0.000       0.022       0.045
sanction_lag2         0.0094      0.008      1.196      0.232      -0.006       0.025
sanction_lag3        -0.0021      0.008     -0.281      0.779      -0.017       0.013
sanction_lag4         0.0006      0.008      0.073      0.942      -0.014       0.016
sanction_lag5         0.0542      0.006      9.664      0.000       0.043       0.065
contig               -0.4399      0.010    -46.251      0.000      -0.459      -0.421
comlang_off          -0.0299      0.005     -6.473      0.000      -0.039      -0.021
colony                0.1707      0.011     15.653      0.000       0.149       0.192
smctry               -0.2500      0.014    -17.684      0.000      -0.278      -0.222
distw                -0.4905      0.002   -296.905      0.000      -0.494      -0.487
=====================================================================================
"""

In [181]:
betas = results.params
betas

GDP_reporter_lag1    0.133425
GDP_reporter_lag2    0.014028
GDP_reporter_lag3    0.016282
GDP_reporter_lag4    0.014240
GDP_reporter_lag5   -0.001125
GDP_partner_lag1     0.037938
GDP_partner_lag2     0.019484
GDP_partner_lag3     0.021123
GDP_partner_lag4     0.002051
GDP_partner_lag5    -0.009324
sanction_lag1        0.033255
sanction_lag2        0.009391
sanction_lag3       -0.002131
sanction_lag4        0.000560
sanction_lag5        0.054189
contig              -0.439925
comlang_off         -0.029910
colony               0.170720
smctry              -0.250011
distw               -0.490493
dtype: float64

In [182]:
# plt.figure(figsize=(8, 5))
# plt.plot(results_threshold["n_columns"], results_threshold["errors"])
# plt.xlabel("Number of columns selected")
# plt.ylabel("RMSE")
# plt.title("Model performance vs sparsity")
# plt.grid()
# plt.show()

In [183]:
betas = gme_model.results_dict["all"].params

for col in rhs_cols:
  test_df.loc[:, col] = pd.to_numeric(test_df[col], errors="coerce")
test_df = test_df.dropna(subset=rhs_cols)
model_columns = betas.index.tolist()

In [184]:
missing_cols = [col for col in model_columns if col not in test_df.columns]
if missing_cols:
  print(f"Warning: The following columns are missing in test_df: {missing_cols}")
  for col in missing_cols:
    test_df[col] = 0.0

In [185]:
X_test = test_df[model_columns].values
beta_values = betas[model_columns].values
linear_combination = X_test @ beta_values
test_df["EXPORT_hat"] = np.exp(linear_combination)

In [186]:
y_true = test_df["EXPORT"].values
y_pred = test_df["EXPORT_hat"].values

print(f"RMSE: {tu.rmse(y_true, y_pred):.4f}")
print(f"MAE: {tu.mae(y_true, y_pred):.4f}")
print(f"RMAE: {tu.rmae(y_true, y_pred):.4f}")
print(f"Pseudo R²: {tu.pseudo_r2(y_true, y_pred):.4f}")
print(f"Within R²: {tu.within_r2(y_true, y_pred, test_df['dyad_id'].values):.4f}")

RMSE: 3.7885
MAE: 3.0832
RMAE: 0.5610
Pseudo R²: 0.4783
Within R²: -22.5117


In [187]:
# Save betas
beta_df = pd.DataFrame(beta_values, index=model_columns, columns=["beta"])
beta_df.to_csv("../../models/ppml_beta_values.csv")